<h2>Initializing</h2>
<h5>Importing dependencies</h5>

In [25]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectFromModel, RFECV, VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from category_encoders import CountEncoder
from xgboost import XGBRegressor

In [2]:
train_data = pd.read_csv('Iowa_train.csv', index_col = 'Id')
test_data = pd.read_csv('Iowa_test.csv', index_col='Id')

x_train, x_val, y_train, y_val = train_test_split(train_data.drop(['SalePrice'], axis=1), 
                                                  train_data['SalePrice'],
                                                  test_size=0.2, random_state=42)

<h2> Data analysis </h2>
<h5> Visualize data and check its consistency</h5>

In [3]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


<h3> Missing values check </h3>

In [4]:
pd.set_option('display.float_format','{:.3%}'.format)
missing = train_data.isnull().sum()
missing_df = pd.DataFrame(data={'count': missing, 'percent': missing / train_data.index.size})
pd.concat([missing_df, pd.DataFrame(data={'count': missing.sum(), 'percent': missing.sum() / np.product(train_data.shape)}, index=['Total'])])

,count,percent
MSSubClass,0,0.000%
MSZoning,0,0.000%
LotFrontage,259,17.740%
LotArea,0,0.000%
Street,0,0.000%
...,...,...
YrSold,0,0.000%
SaleType,0,0.000%
SaleCondition,0,0.000%
SalePrice,0,0.000%


<h5>By seeing the data description, the missing data correspond to features that can be non-present in the real world (a house may not have a pool, fence quality, fireplace nearby, etc), so instead we can just fill it with a 'None' or 'N/A', as its absence will still be informative</h5>

In [5]:
# Get the features with more than 40% of missing values
missing_to_fill = [feature for feature in missing.index if missing_df.loc[feature, 'percent'] > 0.4]
missing_to_fill

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

<h3> Categorical values consistency </h3>
<h5> Now we make sure that all the categorical data have the same unique values (i.e. validation data has no category that is not present in the train data, can also be used in the future for the train data)</h5>

In [6]:
pd.set_option('display.float_format', None)
categorical_cols = [t for t in x_train.columns if x_train[t].dtype == 'object']

def check_consistency(df_test):
    uniques_train = {key:set(x_train[key]) for key in categorical_cols}
    uniques_test = {key:set(df_test[key]) for key in categorical_cols}
    uniques_comparison = [bool(uniques_test[key] - uniques_train[key]) for key in categorical_cols]
    return pd.Series(categorical_cols)[uniques_comparison].to_list()
print('Inconsistent columns: ', check_consistency(x_val))

Inconsistent columns:  ['RoofMatl', 'Electrical']


<h2> Preprocessing data </h2>
<h5> Now we can properly start preprocessing data</h5>

In [7]:
numerical_cols = [col for col in x_train.columns if col not in categorical_cols]
categorical_cols = list(set(categorical_cols) - set(missing_to_fill))

numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                        ('scaler', StandardScaler())])
cat_encoder = CountEncoder(min_group_size=1, handle_unknown=0)
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('encoder', cat_encoder),
                                          ('scaler', StandardScaler())])

NA_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='N/A')),
                                 ('encoder', cat_encoder),
                                 ('scaler', StandardScaler())])
                                 
preprocessing = ColumnTransformer(transformers=[('drop_inconsistent', 'drop', check_consistency),
                                                ('num', numerical_transformer, numerical_cols),
                                                ('na', NA_transformer, missing_to_fill),
                                                ('cat', categorical_transformer, categorical_cols)])

<h3> Feature Selection </h3>
<h5> Since there are a lot of features, it is a good choice to reduce the amount of features to avoid overfitting </h5>

In [20]:
feature_selection = RFECV(LinearRegression(), cv=3)

<h2> Modelling </h2>
<h5> Since we have only a few features, no feature selection will be used, so we jump to modelling. Now we test different estimators to enhace scoring using Cross Validation </h5>

In [26]:
reg = Pipeline(steps=[('prep', preprocessing),
                      ('feat_sel', feature_selection),
                      ('model', XGBRegressor())])

<h3> Evaluation </h3>
<h5> Score the model chosen during Modelling phase (different scorings can be used here)</h5>

In [27]:
reg.fit(x_train, y_train)
reg.score(x_val, y_val)

0.8774325364660688

<h2> Output </h2>
<h5> Make predictions and save properly as a .csv file that can be submitted to Kaggle competitions </h5>

In [23]:
predictions = reg.predict(test_data)

output = pd.DataFrame(data={'Id': test_data.index,
                            'SalePrice': predictions})
output.to_csv('submission.csv', index=False)